In [23]:
import psutil
import subprocess
from time import sleep 

In [ ]:
def connect():
    cmds = ["cd /opt/kafka_2.11-0.9.0.0"
        ,"bin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic topic"
            ,"bin/kafka-console-producer.sh --broker-list localhost:9092 --topic topic"
"bin/kafka-topics.sh --list --zookeeper localhost:2181"]
    try:
        for cmd in cmds:
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
            print('Command executed successfully.')
            print('Output:\n', result.stdout)
        
    except subprocess.CalledProcessError as ex:
        print('Command execution failed.')
        print('Error:', ex.stderr)

In [4]:
def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor


def get_stats():
    msg={}
    STRUMENT='CPU '
    msg[STRUMENT+'Physical cores']=psutil.cpu_count(logical=False)
    msg[STRUMENT+'Total cores']=psutil.cpu_count(logical=True)
    for i, percentage in enumerate(psutil.cpu_percent(percpu=True, interval=1)):
        msg[STRUMENT+'Core'+str(i)]= percentage

    STRUMENT='RAM '
    svmem = psutil.virtual_memory()
    msg[STRUMENT+'Total']=get_size(svmem.total)
    msg[STRUMENT+'Available']=get_size(svmem.available)
    msg[STRUMENT +'Used']=get_size(svmem.used)
    msg[STRUMENT +'Percentage']=svmem.percent
    swap = psutil.swap_memory()
    STRUMENT='RAM SWAP'
    msg[STRUMENT+'Total']=get_size(swap.total)
    msg[STRUMENT+'Free']=get_size(swap.free)
    msg[STRUMENT+'Used']=get_size(swap.used)
    msg[STRUMENT+'Percentage']=swap.percent
    STRUMENT=str('DISK  ')
    disk_io = psutil.disk_io_counters()
    msg[STRUMENT+'Total read']=get_size(disk_io.read_bytes)
    msg[STRUMENT+'Total write']=get_size(disk_io.write_bytes)

    return str(msg)

2934


In [21]:
connect()
i = 0
while(1):
    cmd_prod = "bin/kafka-console-producer.sh --broker-list localhost:9092 --topic topic "
    cmd_full = "echo "+ '\"'+get_stats()+'\" | ' + cmd_prod
    cmd_mess = subprocess.run(cmd_full, shell=True, capture_output=True, text=True)
    print("Message {i} sent")
    sleep(1)
    i+=1